In [ ]:
# jupyter-notebook 

import cv2
import numpy as np
from matplotlib import pyplot as plt
import itertools  
import time
import numba as nb
import copy
from components_container import ComponentsContainerWithHistory
import logging
logging.basicConfig(level=logging.INFO)
from skimage import io
%matplotlib inline
from skimage.morphology import binary_dilation, reconstruction
from skimage.measure import label,regionprops_table
from scipy import ndimage
import pandas as pd

In [ ]:
#!pip install opencv-python numba


In [ ]:
displayed_sq = 1500
width_margin = 0
height_margin = 0
def show_img(img):
    fig = plt.figure(figsize=(13, 13), dpi= 80, facecolor='w', edgecolor='k')
    plt.imshow(img[height_margin:height_margin+displayed_sq,width_margin:width_margin+displayed_sq], cmap="ocean")

In [ ]:
INimg = io.imread('../../notebook_plaque/3155_Plaque.tif', plugin='tifffile')

INimg = np.moveaxis(INimg,0,1)

#img = img[:2500,:2500]

In [ ]:
INimg.shape

In [ ]:
def draw_grayscale(container):
    labs = sorted(np.unique(container.markers)[1:], key = lambda x: -container.components_index.get(x).size)
    print(container.markers.shape)
    print(len(labs))
    pic = np.zeros(container.markers.shape)
    mask_color = 1
    for mask_lab in labs:
        pic[container.markers == mask_lab] = mask_color#np.sqrt(container.components_index.get(nucl_lab).size)
        mask_color += 1
    
    fig = plt.figure(figsize=(11, 11), dpi=80, facecolor='w', edgecolor='k')
    
    plt.imshow(pic[height_margin:height_margin + displayed_sq, width_margin:width_margin + displayed_sq],
                   cmap="Set1")
    print(mask_color)
    return pic, mask_color

def computeMask(img, BACKGROUND_INTENSITY_THRESHOLD=10, MIN_SIZE=100,show=False):
    #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = img.copy()
    invert = cv2.bitwise_not(gray)
    if show:
        show_img(invert)
    blur = cv2.GaussianBlur(img,(5,5),0)
    ret, binary = cv2.threshold(blur, BACKGROUND_INTENSITY_THRESHOLD, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    kernel = np.ones((1, 1), 'uint8')

    binary = cv2.dilate(binary, kernel, iterations=5)
    if show:
        show_img(binary)
    # filter all noise using minimum size threshold
    container = ComponentsContainerWithHistory(binary, MIN_SIZE, logging.INFO)
    if show:
        show_img(container.markers)
    return draw_grayscale(container)


def enhance_img_with_container(img, container):
    edge_color = [80,0,80]
    hull_color = [0,0,100]
    for nucl_lab in container.nucleus_labs:           
        hull = np.array(container.components_index.get(nucl_lab).hull, np.int32)
        hull.reshape((-1,1,2))
        hull = hull[:,::-1]
        cv2.polylines(img, [hull],True, hull_color)

        for ed in container.components_index.get(nucl_lab).edge:
            img[ed[0], ed[1]] = edge_color
    return img



def saveMask(outFile,oMask):
    #io.imsave(, outMask.astype('uint32'), plugin='tifffile')
    #convert to binary
    glNew = oMask>0
    #fill holes
    #dilate it 'dil' times
    for i in range(1):
        glNew = binary_dilation(glNew)
    glNew = ndimage.binary_fill_holes(glNew)

    # label image regions
    glNew = label(glNew)
    #print(glNew.shape)
    regionprops_table(glNew,properties=['label','area'])
    io.imsave(outFile,  np.moveaxis(glNew,0,1).astype('uint32'), plugin='tifffile')

In [ ]:
for fid in ['1796','2997','3155','3196','3280']:
    INimg = io.imread(f'../../notebook_plaque/train/{fid}_Plaque.tif', plugin='tifffile')
    INimg = np.moveaxis(INimg,0,1)
    outFile = f'Plaques/{fid}_Plaque'
    outMask  =   np.zeros((INimg.shape[0],INimg.shape[1]))
    numCells = 0
    #outFile = '2997_Glia6M'
    w = INimg.shape[0]
    h = INimg.shape[1]
    TILE_SIZE = 7500
    xNum = INimg.shape[0]//TILE_SIZE 
    yNum = INimg.shape[1]//TILE_SIZE
    print(w,h,TILE_SIZE, xNum, yNum)
    for ix in range(xNum+1):
        for iy in range(yNum+1):
            xLim = (ix*TILE_SIZE,min(w,(ix+1)*TILE_SIZE))
            yLim = (iy*TILE_SIZE,min(h,(iy+1)*TILE_SIZE))
            print("DIMS....",ix,iy,xLim,yLim)
            maskImg, Ncomp = computeMask(INimg[xLim[0]:xLim[1],yLim[0]:yLim[1]], show=False)
            print(Ncomp)
            outMask[xLim[0]:xLim[1],yLim[0]:yLim[1]] = maskImg
            numCells += Ncomp


    saveMask('../'+outFile+'.tif', outMask)
